# Closed loop simulations

The objective in this notebook is to close the former strategy in Symuvia. 

In [1]:
import os

from ctypes import cdll, create_string_buffer, c_int, byref, c_bool, c_double

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

from xmltodict import parse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

from symfunc import queueveh, getlead, getspace, getleaderspeed, updatelist, typedict
from contfunc import reversedEnumerate, find_idx_ldr, initial_setup_mpc, 

Parameters

In [14]:
DT = 0.1 # Sample time 

KC = 0.16 # CAV max density 
KH = 0.0896 # HDV max density
VF = 25.0 # Speed free flow
W = 6.25 # Congestion speed 
E  = 25.0*0.3 # Speed drop for relaxation 

GCAV = 1/(KC*W) # Time headway CAV 
GHDV = 1/(KH*W) # Time headway HDV 
SCAV = VF/(KC*W)+1/KC #  Desired space headway CAV 
SHDV = VF/(KH*W)+1/KH #  Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5 # Max. Acceleration
U_MIN = -1.5 # Min. Acceleration

Appending to contfunc.py


Database engine configuration

In [3]:
dir_path = os.getcwd()
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

Load traffic library

In [4]:
dir_path = os.getcwd()
lib_path_name = ('..','SymuviaModPos','Contents','Frameworks','libSymuVia.dylib')
full_name = os.path.join(dir_path,*lib_path_name)
symuvialib = cdll.LoadLibrary(full_name)     

Load traffic network 

In [6]:
file_path = ('..', 'Network', 'Merge.xml')
file_name = os.path.join(dir_path, *file_path)

# Pointers
sRequest = create_string_buffer(100000)
bEnd = c_int()
bSecond = c_bool(True)

Widgets

In [7]:
progressSim = widgets.FloatProgress(
    value=5,
    min=0,
    max=71.9,
    step=0.1,
    description='Simulating:',
    bar_style='info',
    orientation='horizontal'
)
tiVal = widgets.BoundedFloatText(
    value=7.5,
    min=0,
    max=80.0,
    step=0.1,
    description='Time step:',
    disabled=False
)

Loop simulation

In [10]:
%%time

N = 13 # Simulation steps

# Start simulation from beginning
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))

step = iter(range(N)) 

t = []

display(progressSim) 
display(tiVal)

bSuccess = 2 
while bSuccess>0:
    bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
    try:
        next(step)        
        dParsed = parse(sRequest.value.decode('UTF8'))
        ti = dParsed['INST']['@val']
        if dParsed['INST']['TRAJS'] is None:
            pass 
            #print('No vehicles in the network at time: {}'.format(ti))
        else:            
            lVehOD = dParsed['INST']['TRAJS']['TRAJ']      
            lTrajVeh = []
            try:
                lTrajVeh = typedict(lVehOD)
                lTrajVeh['ti'] = ti
                dLeader = {lTrajVeh['tron']: [lTrajVeh['id']]}
                lTrajVeh['ldr'] = getlead(dLeader, lTrajVeh)                  
            except TypeError:
                # Multiple veh @ ti
                for i,  veh in enumerate(lVehOD):
                    TrajVeh = typedict(veh)
                    TrajVeh['ti'] = ti
                    dLeader = queueveh(dLeader, TrajVeh)
                    TrajVeh['ldr'] = getlead(dLeader, TrajVeh)
                    lTrajVeh.append(TrajVeh)
            lSpc = getspace(lTrajVeh)
            lLdrV = getleaderspeed(lTrajVeh)
            lTrajVeh = updatelist(lTrajVeh,lSpc)
            lTrajVeh = updatelist(lTrajVeh,lLdrV)
            # print('{} vehicles in the network at time: {}'.format(results.rowcount, ti))
          
        t.append(ti)
        progressSim.value = ti
        tiVal.value = ti
    except StopIteration:
        print('Stop by iteration')
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
    except:
        print(i)
        bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
        print('Return from Symuvia Empty: {}'.format(sRequest.value.decode('UTF8')))
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
        

FloatProgress(value=71.9, bar_style='info', description='Simulating:', max=71.9)

BoundedFloatText(value=80.0, description='Time step:', max=80.0, step=0.1)

Stop by iteration
Last simluation step at time: 1.30
CPU times: user 141 ms, sys: 7.89 ms, total: 149 ms
Wall time: 146 ms


In [9]:
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))
N = 800 # Simulation steps
for s in time:
    
    # For all vehicle connected if created:
    # Drive vehicle 
   # symuvialib.SymDriveVehiculeEx(Id,sTroncon, nVoie, dPos, bForce)

    bResult = symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd)) 
    dParsed = parse(sRequest.value.decode('UTF8'))
    print('Instant:',s+1)


#     print(f'bResult: {bResult}')    
#     print(f'OutputNet: {sRequest.value}')        
    ti = dParsed['INST']['@val']
    print(ti, bResult)

# # Create vehicle at time step 0.5
# stype = 'CAV'
# sOrigin= 'Ext_In_main'
# sDestination = 'Ext_Out_main'
# nVoie = c_int(1)
# dbTime = c_double(0.05)
# nIdVeh = symuvialib.SymCreateVehicleEx(stype.encode('UTF8'), sOrigin.encode('UTF8'), sDestination.encode('UTF8'), nVoie, dbTime)
# print('Vehicle created', nIdVeh)
    
    
# # Run time step 
# bResult = symuvialib.SymRunNextStepEx(sRequest, 1, byref(bEnd)) 
# dParsed = parse(sRequest.value.decode('UTF8'))
# print(f'bResult: {bResult}')    
# print(f'OutputNet: {sRequest.value}')  
# ti = dParsed['INST']['@val']
# print(ti, bResult)   

# # Drive vehicle @ 0.5

sTroncon = 'In_main'.encode('UTF8')
nVoie = c_int(1)
dPos = c_double(15.0)
Id = c_int(0)
bForce = c_int(1)
#print(sTroncon,nVoie,dPos,Id, bForce)
dParsed1 = dParsed
print(dParsed1)

nres = symuvialib.SymDriveVehicleEx(Id, sTroncon, nVoie, dPos, bForce)


bResult = symuvialib.SymRunNextStepEx(sRequest, 1, byref(bEnd)) 
dParsed = parse(sRequest.value.decode('UTF8'))
# print(f'bResult: {bResult}')    
# print(f'OutputNet: {sRequest.value}')  
ti = dParsed['INST']['@val']
print(ti, bResult)   
print(dParsed)

# print('Drive ',nres )

NameError: name 'time' is not defined